In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig

In [2]:
# This recipe requires a HuggingFace token to access the mistral models from the hub. You will need to create a HuggingFace login
# and generate a token for use at https://huggingface.co/settings/tokens. You will also need to visit the model card at 
# https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3 and accept the terms to use the model. Once you have generated the token,
# save it in an environment variable named HUGGINGFACE_TOKEN and read it in the recipe as shown in the snippet below.
from huggingface_hub import login
hf_token = os.environ.get('HUGGINGFACE_TOKEN')
login(token=hf_token)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_use_double_quant=True,
                                         bnb_4bit_quant_type= "nf4"
                                         )
model = AutoModelForCausalLM.from_pretrained(
            "mistralai/Mistral-7B-v0.3", 
            device_map="auto", 
            quantization_config=quantization_config,
        )

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3", padding_side="left")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
generation_config = GenerationConfig(
    num_beams=4,
    early_stopping=True,
    eos_token_id=model.config.eos_token_id,
    pad_token_id=model.config.eos_token_id,
    max_new_tokens=900,
)

In [5]:
seed_sentence = "Step by step way on how to make an apple pie:"

In [6]:
model_inputs = tokenizer([seed_sentence], return_tensors="pt").to(device)
generated_ids = model.generate(**model_inputs, generation_config=generation_config)

In [7]:
generated_tokens = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_tokens)

Step by step way on how to make an apple pie:

1. Preheat the oven to 350 degrees Fahrenheit.
2. Peel and core the apples.
3. Cut the apples into thin slices.
4. Place the apples in a large bowl.
5. Add the sugar, cinnamon, and nutmeg to the apples.
6. Stir the apples until they are evenly coated with the sugar and spices.
7. Pour the apples into a pie dish.
8. Place the pie dish on a baking sheet.
9. Bake the pie for 45 minutes to 1 hour, or until the apples are soft and the crust is golden brown.
10. Remove the pie from the oven and let it cool for 10 minutes before serving.

## How do you make an apple pie from scratch?

To make an apple pie from scratch, you will need the following ingredients:

- 2 cups of all-purpose flour
- 1 teaspoon of salt
- 1/2 cup of shortening
- 1/2 cup of cold water
- 4 cups of peeled, cored, and sliced apples
- 1 cup of sugar
- 1 teaspoon of cinnamon
- 1/4 teaspoon of nutmeg
- 1/4 teaspoon of allspice
- 2 tablespoons of cornstarch
- 1 tablespoon of lemon